## Нейросеть для California housing

Будем работать с набором данным `fetch_california_housing` из `sklearn`.

In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd


data = fetch_california_housing()

X = pd.DataFrame(data['data'], columns=data['feature_names'])
y = data['target']

X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [2]:
y

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

#### 1.1. Разбейте данные на обучение и тест

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X.values, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape

((16512, 8), (4128, 8))

#### 1.2. Создайте архитектуру для этой задачи

In [4]:
from keras.layers import Dense
from keras.models import Sequential


model = Sequential([
    Dense(4, input_shape=(8,), activation='relu'),
    Dense(1, activation='linear')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 36        
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


#### 1.3. Скомпилируйте сеть

Подберите нужную функцию потерь и метрику. Оптимизатор можете использовать 'sgd'

In [5]:
from keras.optimizers import SGD
INIT_LR = 0.01
opt = SGD(learning_rate=INIT_LR)
model.compile(optimizer=opt, loss='mse', metrics='mae')

#### 1.4. Обучите нейросеть

Выберите нужное количество эпох.


In [6]:
%%time
EPOCHS = 200
H = model.fit(X_train, Y_train, epochs=EPOCHS, batch_size=1000, validation_data=(X_test, Y_test))

Epoch 1/200
17/17 [==============================] - 1s 16ms/step - loss: 214563234529673216.0000 - mae: 114295112.0000 - val_loss: 494137866452992.0000 - val_mae: 22229206.0000
Epoch 2/200
17/17 [==============================] - 0s 4ms/step - loss: 367831199776768.0000 - mae: 19090694.0000 - val_loss: 248618275569664.0000 - val_mae: 15767631.0000
Epoch 3/200
17/17 [==============================] - 0s 4ms/step - loss: 185069067042816.0000 - mae: 13541426.0000 - val_loss: 125089211219968.0000 - val_mae: 11184326.0000
Epoch 4/200
17/17 [==============================] - 0s 4ms/step - loss: 93115268464640.0000 - mae: 9605232.0000 - val_loss: 62937016303616.0000 - val_mae: 7933283.0000
Epoch 5/200
17/17 [==============================] - 0s 5ms/step - loss: 46849763311616.0000 - mae: 6813200.5000 - val_loss: 31665961304064.0000 - val_mae: 5627250.5000
Epoch 6/200
17/17 [==============================] - 0s 5ms/step - loss: 23571833290752.0000 - mae: 4832747.5000 - val_loss: 1593231749939

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scattergl(y=H.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=H.history['val_loss'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')

fig.show()

In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scattergl(y=H.history['mae'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=H.history['val_mae'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')

fig.show()

In [8]:
from sklearn.metrics import mean_absolute_error
Y_predict = model.predict(X_test)
print(f'Средняя ошибка: ${int(mean_absolute_error(Y_predict, Y_test)*100000)}')

129/129 [==============================] - 0s 1ms/step
Средняя ошибка: $90722


#### 1.5. Сделайте вывод, хорошо ли модель обучается?

Если модель обучается плохо, можете внести изменения в работу.

In [46]:
from sklearn.preprocessing import MinMaxScaler


model = Sequential([
    Dense(256, input_shape=(8,), activation='relu'),
    Dense(128, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

# Нормализуем данные
mms = MinMaxScaler()
X_norm = mms.fit_transform(X_train)
X_test_norm = mms.fit_transform(X_test)

INIT_LR = 0.01
opt = SGD(learning_rate=INIT_LR)
model.compile(optimizer=opt, loss='mse', metrics='mae')

P.S.: Сеть скорее всего обучается плохо, так как данные не масштабированы.

In [47]:
%%time
EPOCHS = 200
H = model.fit(X_norm, Y_train, epochs=EPOCHS, batch_size=1000, validation_data=(X_test_norm, Y_test))

Epoch 1/200
17/17 [==============================] - 1s 25ms/step - loss: 2.6830 - mae: 1.2346 - val_loss: 1.2845 - val_mae: 0.8751
Epoch 2/200
17/17 [==============================] - 0s 15ms/step - loss: 1.2813 - mae: 0.8883 - val_loss: 1.2298 - val_mae: 0.8854
Epoch 3/200
17/17 [==============================] - 0s 12ms/step - loss: 1.2335 - mae: 0.8797 - val_loss: 1.1889 - val_mae: 0.8687
Epoch 4/200
17/17 [==============================] - 0s 13ms/step - loss: 1.1925 - mae: 0.8661 - val_loss: 1.1493 - val_mae: 0.8491
Epoch 5/200
17/17 [==============================] - 0s 13ms/step - loss: 1.1496 - mae: 0.8483 - val_loss: 1.1101 - val_mae: 0.8452
Epoch 6/200
17/17 [==============================] - 0s 14ms/step - loss: 1.1049 - mae: 0.8345 - val_loss: 1.0644 - val_mae: 0.8173
Epoch 7/200
17/17 [==============================] - 0s 12ms/step - loss: 1.0579 - mae: 0.8145 - val_loss: 1.0181 - val_mae: 0.8021
Epoch 8/200
17/17 [==============================] - 0s 13ms/step - loss: 1.

#### 1.6. Сделайте предсказания на тестовых данных

In [49]:
%matplotlib inline
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scattergl(y=H.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=H.history['val_loss'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')

fig.show()

In [50]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scattergl(y=H.history['mae'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=H.history['val_mae'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')

fig.show()

#### 1.7. Посчитайте метрику качества на тестовых данных

In [51]:
from sklearn.metrics import mean_absolute_error
Y_predict = model.predict(X_test_norm)
print(f'Средняя ошибка: ${int(mean_absolute_error(Y_predict, Y_test)*100000)}')

129/129 [==============================] - 0s 1ms/step
Средняя ошибка: $49896
